In [1]:
import pandas as pd
import ast
from tqdm import tqdm
tqdm.pandas()

In [2]:
df_yake_claims = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/Yake/g_patent_claims_cleantech_yake.json')

In [4]:
# Iterate over rows in dataframe
for index, row in tqdm(df_yake_claims.iterrows()):
    # Cast row 'keywords_yake' column to string
    row['keywords_yake'] = str(row['keywords_yake'])
    # Check if keywords_yake column starts with "[[[" and ends with "]]]""
    if row['keywords_yake'].startswith('[[[') and row['keywords_yake'].endswith(']]]'):
        # Remove first "[" and last "]" from keywords_yake column
        row['keywords_yake'] = row['keywords_yake'][1:-1]
    # Cast row 'keywords_yake' column to list
    row['keywords_yake'] = ast.literal_eval(row['keywords_yake'])
    # Assign modified 'keywords_yake' list to temporary variable
    keywords_yake_temp = row['keywords_yake']
    # Assign temporary variable to 'keywords_yake' column
    df_yake_claims.at[index, 'keywords_yake'] = keywords_yake_temp

515742it [01:11, 7198.08it/s]


In [5]:
df_yake_claims['keywords_yake_exploded'] = ''

# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Check if 'keywords_yake' column is not a list
    if not isinstance(row['keywords_yake'], list):
        continue
    # Check if 'keywords_yake' column is an empty list or contains only empty lists
    if not any(keyword for keyword in row['keywords_yake']):
        continue
    # Iterate over keywords in 'keywords_yake' column and append to 'keywords_yake_exploded' column
    else:
        for keyword in row['keywords_yake']:
            df_yake_claims.at[index, 'keywords_yake_exploded'] += keyword[0] + ', '

515742it [02:15, 3809.49it/s]


In [8]:
# Cast 'keywords_yake_exploded' column to list
df_yake_claims['keywords_yake_exploded'] = df_yake_claims['keywords_yake_exploded'].str.split(', ')

In [11]:
# Delete last item in 'keywords_yake_exploded' cells
df_yake_claims['keywords_yake_exploded'] = df_yake_claims['keywords_yake_exploded'].apply(lambda x: x[:-1])

# Postprocessing

In [13]:
keywords_list = []
# Iterate over rows in dataframe
for index,row in tqdm(df_yake_claims.iterrows()):
    # Append 'keywords_yake_exploded' to keywords_list
    keywords_list += row['keywords_yake_exploded']

515742it [00:12, 40277.88it/s]


In [14]:
# Generate unique list of keywords
keywords_list_unique = list(set(keywords_list))

In [46]:
df_keywords_list_unique = pd.DataFrame(keywords_list_unique, columns=['keyword_yake'])

# Add empty columns abs_frequency and doc_frequency
# df_keywords_list_unique['abs_frequency'] = 0
# df_keywords_list_unique['doc_frequency'] = 0

In [47]:
from collections import Counter

# Assume df_yake_claims and df_keywords_list_unique are your DataFrames

# Step 1: Flatten the list of keywords into a single list
flattened_keywords = [keyword for keyword_list in tqdm(df_yake_claims['keywords_yake_exploded']) for keyword in keyword_list]

# Step 2: Use collections.Counter to count the occurrences of each keyword
keyword_counts = Counter(flattened_keywords)

# Step 3: Convert the keyword counts to a DataFrame
count_df = pd.DataFrame(keyword_counts.items(), columns=['keyword_yake', 'abs_frequency'])

# Step 4: Merge the count_df with df_keywords_list_unique on the 'keyword_yake' column
df_keywords_list_unique = df_keywords_list_unique.merge(count_df, on='keyword_yake', how='left')

# If you want to fill NaN values with 0 (in case some keywords didn't have counts)
# df_keywords_list_unique['abs_frequency'] = df_keywords_list_unique['abs_frequency'].fillna(0).astype(int)

  0%|          | 0/515742 [00:00<?, ?it/s]

100%|██████████| 515742/515742 [00:00<00:00, 2482500.26it/s]


In [49]:
### MAYBE I NEED TO REDO THIS WITH FULL TEXT CLAIMS INSTEAD OF JUST THE YAKE KEYWORDS

# Calculate document frequency by dividing absolute frequency by len of df_yake_claims
df_keywords_list_unique['doc_frequency'] = df_keywords_list_unique['abs_frequency'] / len(df_yake_claims)

In [107]:
# Prune dataframe by document frequency and absolute frequency
min_abs_frequency = 10
max_doc_frequency = 0.3

df_keywords_list_unique_pruned = df_keywords_list_unique[(df_keywords_list_unique['abs_frequency'] >= min_abs_frequency) & (df_keywords_list_unique['doc_frequency'] <= max_doc_frequency)]

In [108]:
# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [109]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
# nltk.download('stopwords')

# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# Now use .loc to modify the 'keyword_yake' column
df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stopwords.words('english'))])
)

In [111]:
# Only consider n-grams with n >= 2
min_ngram_length = 2

# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

def filter_keywords(keyword_string):
    # Split the keyword string into individual keywords
    keywords = keyword_string.split(', ')
    # Filter out keywords with fewer than 2 words
    filtered_keywords = [keyword for keyword in keywords if len(keyword.split()) >= min_ngram_length]
    # Join the filtered keywords back into a string
    return ', '.join(filtered_keywords)

# Create a copy of df_keywords_list_unique_pruned to avoid SettingWithCopyWarning
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned.copy()

# Now use .loc to modify the 'keyword_yake' column
df_keywords_list_unique_pruned.loc[:, 'keyword_yake'] = df_keywords_list_unique_pruned['keyword_yake'].apply(filter_keywords)

In [112]:
# Delete rows with empty 'keyword_yake' column
df_keywords_list_unique_pruned = df_keywords_list_unique_pruned[df_keywords_list_unique_pruned['keyword_yake'] != '']

In [113]:
# Reset index
df_keywords_list_unique_pruned.reset_index(drop=True, inplace=True)

In [114]:
# Sort descending by absolute frequency
df_keywords_list_unique_pruned.sort_values(by=['abs_frequency'], ascending=False, inplace=True)

In [115]:
### MAYBE I NEED TO REDO THIS PART WITH THE CLAIM TEXTS INSTEAD OF THE KEYWORDS

import spacy

# Download spacy model
nlp = spacy.load('en_core_web_sm')

# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned.copy()

# Perform part-of-speech tagging on the 'keyword_yake' column 
# and save the POS tags in a new column 'keyword_yake_pos'
df_keywords_list_unique_pruned_pos['keyword_yake_pos'] = df_keywords_list_unique_pruned_pos['keyword_yake'].progress_apply(
    lambda x: [token.pos_ for token in nlp(x)]
)

100%|██████████| 24789/24789 [00:59<00:00, 414.57it/s]


In [116]:
# Filter out all keywords that do not contain NOUN, PRON or PROPN in their POS tags
df_keywords_list_unique_pruned_pos = df_keywords_list_unique_pruned_pos[df_keywords_list_unique_pruned_pos['keyword_yake_pos'].apply(
    lambda x: any(pos in ['NOUN', 'PRON', 'PROPN'] for pos in x)
)]

In [117]:
# Reset index
df_keywords_list_unique_pruned_pos.reset_index(drop=True, inplace=True)

# Keyword Embedding

In [118]:
from sentence_transformers import SentenceTransformer
import torch

In [119]:
model_climatebert = SentenceTransformer('climatebert/distilroberta-base-climate-f')
model_bertforpatents = SentenceTransformer('anferico/bert-for-patents')
model_patentsberta = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/anferico_bert-for-patents. Creating a new one with MEAN pooling.


In [120]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available: {}".format(torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU available: NVIDIA RTX A4500


In [121]:
# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_embeddings = df_keywords_list_unique_pruned_pos.copy()

# Perform sentence embedding on the 'keyword_yake' column
df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding'] = df_keywords_list_unique_pruned_embeddings['keyword_yake'].progress_apply(
    lambda x: model_climatebert.encode(x)
)

df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding'] = df_keywords_list_unique_pruned_embeddings['keyword_yake'].progress_apply(
    lambda x: model_bertforpatents.encode(x)
)

df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding'] = df_keywords_list_unique_pruned_embeddings['keyword_yake'].progress_apply(
    lambda x: model_patentsberta.encode(x)
)

100%|██████████| 23742/23742 [02:45<00:00, 143.64it/s]


# Dimensionality Reduction

In [122]:
import umap
import numpy as np
reducer = umap.UMAP(random_state=42)

In [123]:
# Assuming each element in the embedding columns is a 1D array
# Stack all the embeddings into a 2D array
embeddings_climatebert = np.vstack(df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding'].values)
embeddings_bertforpatents = np.vstack(df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding'].values)
embeddings_patentsberta = np.vstack(df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding'].values)

# Perform UMAP dimensionality reduction
umap_embeddings_climatebert = reducer.fit_transform(embeddings_climatebert)
umap_embeddings_bertforpatents = reducer.fit_transform(embeddings_bertforpatents)
umap_embeddings_patentsberta = reducer.fit_transform(embeddings_patentsberta)

# Assign the reduced dimension embeddings back to new DataFrame columns
df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding_umap'] = list(umap_embeddings_climatebert)
df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding_umap'] = list(umap_embeddings_bertforpatents)
df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding_umap'] = list(umap_embeddings_patentsberta)

In [124]:
# Split the UMAP coordinates into separate columns
df_keywords_list_unique_pruned_embeddings[['climatebert_x', 'climatebert_y']] = pd.DataFrame(df_keywords_list_unique_pruned_embeddings['keyword_yake_climatebert_embedding_umap'].tolist(), index=df_keywords_list_unique_pruned_embeddings.index)
df_keywords_list_unique_pruned_embeddings[['bertforpatents_x', 'bertforpatents_y']] = pd.DataFrame(df_keywords_list_unique_pruned_embeddings['keyword_yake_bertforpatents_embedding_umap'].tolist(), index=df_keywords_list_unique_pruned_embeddings.index)
df_keywords_list_unique_pruned_embeddings[['patentsberta_x', 'patentsberta_y']] = pd.DataFrame(df_keywords_list_unique_pruned_embeddings['keyword_yake_patentsberta_embedding_umap'].tolist(), index=df_keywords_list_unique_pruned_embeddings.index)

# Clustering

In [125]:
# Import HDBSCAN
import hdbscan

In [139]:
# Ensure you're working on a new DataFrame, not a slice of an old one
df_keywords_list_unique_pruned_embeddings_hdbscan = df_keywords_list_unique_pruned_embeddings.copy()

# Perform HDBSCAN clustering on the UMAP coordinates
# clusterer_climatebert = hdbscan.HDBSCAN(min_cluster_size=250, min_samples=1).fit(df_keywords_list_unique_pruned_embeddings_hdbscan[['climatebert_x', 'climatebert_y']])
# clusterer_bertforpatents = hdbscan.HDBSCAN(min_cluster_size=250, min_samples=1).fit(df_keywords_list_unique_pruned_embeddings_hdbscan[['bertforpatents_x', 'bertforpatents_y']])
clusterer_patentsberta = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=1).fit(df_keywords_list_unique_pruned_embeddings_hdbscan[['patentsberta_x', 'patentsberta_y']])

# Assign the cluster labels back to the DataFrame
# df_keywords_list_unique_pruned_embeddings_hdbscan['climatebert_cluster'] = clusterer_climatebert.labels_
# df_keywords_list_unique_pruned_embeddings_hdbscan['bertforpatents_cluster'] = clusterer_bertforpatents.labels_
df_keywords_list_unique_pruned_embeddings_hdbscan['patentsberta_cluster'] = clusterer_patentsberta.labels_

# Plotting

In [140]:
import plotly.express as px

In [141]:
# Filter out data points belonging to cluster -1
# df_filtered_climatebert = df_keywords_list_unique_pruned_embeddings_hdbscan[df_keywords_list_unique_pruned_embeddings_hdbscan['climatebert_cluster'] != -1]
# df_filtered_bertforpatents = df_keywords_list_unique_pruned_embeddings_hdbscan[df_keywords_list_unique_pruned_embeddings_hdbscan['bertforpatents_cluster'] != -1]
df_filtered_patentsberta = df_keywords_list_unique_pruned_embeddings_hdbscan[df_keywords_list_unique_pruned_embeddings_hdbscan['patentsberta_cluster'] != -1]

In [142]:
# Plot the UMAP embeddings in three figures, color coded by cluster
# fig_climatebert = px.scatter(df_filtered_climatebert, x='climatebert_x', y='climatebert_y', color='climatebert_cluster', hover_data=['keyword_yake'], title='UMAP projection of the YAKE keywords (climatebert)')
# fig_bertforpatents = px.scatter(df_filtered_bertforpatents, x='bertforpatents_x', y='bertforpatents_y', color='bertforpatents_cluster', hover_data=['keyword_yake'], title='UMAP projection of the YAKE keywords (bertforpatents)')
fig_patentsberta = px.scatter(
    df_filtered_patentsberta, 
    x='patentsberta_x', 
    y='patentsberta_y', 
    color='patentsberta_cluster', 
    hover_data=['keyword_yake'], 
    title='UMAP projection of the YAKE keywords (patentsberta)',
    height=600,  # Adjust as needed
    width=800    # Adjust as needed
)
# Display the figures
# fig_climatebert.show()
# fig_bertforpatents.show()
fig_patentsberta.show()

# Name Cluster

In [143]:
# Build dataframe of keywords and their corresponding clusters, where each row is a cluster with list of keywords
# df_keywords_clusters_climatebert = df_keywords_list_unique_pruned_embeddings_hdbscan.groupby('climatebert_cluster')['keyword_yake'].apply(list).reset_index(name='keywords')
# df_keywords_clusters_bertforpatents = df_keywords_list_unique_pruned_embeddings_hdbscan.groupby('bertforpatents_cluster')['keyword_yake'].apply(list).reset_index(name='keywords')
df_keywords_clusters_patentsberta = df_keywords_list_unique_pruned_embeddings_hdbscan.groupby('patentsberta_cluster')['keyword_yake'].apply(list).reset_index(name='keywords')

In [147]:
from transformers import pipeline, set_seed

In [150]:
# Initialize a text-generation pipeline with GPT-3
generator = pipeline('text-generation', model='mistralai/Mistral-7B-v0.1')
set_seed(42)

KeyError: 'mistral'

In [ ]:
def generate_cluster_name(keywords):
    # Create a prompt from the keywords
    prompt = f"Generate a name for a cluster containing the following keywords: {keywords}. The name shall be 2-3 words long and shall contain at least one noun."
    # Generate a response using the GPT-3 model
    response = generator(prompt, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text']
    # Extract the cluster name from the response
    # Assume the name is the last word or phrase in the generated text
    cluster_name = response.split()[-1]
    return cluster_name

In [ ]:
# Apply the function to the 'keywords' column to generate cluster names
df['cluster_name'] = df['keywords'].apply(generate_cluster_name)